# Catalogue pre-processing

## Importing libraries

In [1]:
%matplotlib inline
import os
import re
import sys

#sys.path.append('./../../../../hmtk/')
#sys.path.append('./../../../../oq-hazardlib/')
#sys.path.append('/home/openquake/GEM/oq-mbt/')

import h5py
import numpy
import subprocess
import cPickle as pickle
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

from mpl_toolkits.basemap import Basemap
from copy import deepcopy

from hmtk.seismicity.occurrence.weichert import Weichert
from hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971 
from hmtk.plotting.seismicity.completeness.plot_stepp_1972 import create_stepp_plot
from hmtk.plotting.seismicity.catalogue_plots import (plot_magnitude_time_scatter, 
                                                      plot_magnitude_time_density)
from hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from hmtk.seismicity.selector import CatalogueSelector
from hmtk.seismicity.declusterer.dec_gardner_knopoff import GardnerKnopoffType1
from hmtk.seismicity.declusterer.distance_time_windows import GardnerKnopoffWindow

ImportError: No module named hmtk.seismicity.occurrence.weichert

In [4]:
from metys import g_prj

# -- Configuration parameters --
cutoff_magnitude = float(g_prj.mod['ASC']['catalogue_cutoff_magnitude'])
binwidth = float(g_prj.mod['ASC']['mfd_binwidth'])
print 'MFD bin width    : %.2f' % (binwidth)
print 'Magnitude cutoff : %.2f' % (cutoff_magnitude)

# -- Load data --
catalogue = g_prj.mod['ASC']['catalogue_from_csv']

ImportError: No module named metys

## Main settings

## Read catalogue
Read the catalogue from the .csv file and remove events below the cutoff magnitude.

In [ ]:
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)
selector.within_depth_range(300, 0)

In [ ]:
print numpy.min(catalogue.data['depth'])
print numpy.max(catalogue.data['depth'])

## Catalogue declustering

In [ ]:
distance_time_wind = GardnerKnopoffWindow()
config = {'time_distance_window': distance_time_wind, 'fs_time_prop': .9}
declusterer = GardnerKnopoffType1()
vcl, flag = declusterer.decluster(catalogue, config)
catalogue_original = deepcopy(catalogue)
print 'Original number of events', catalogue_original.get_number_events()
catalogue.select_catalogue_events(numpy.where(flag == 0)[0])
print 'Number of mainshocks:', len(catalogue.data['magnitude'])

## Catalogue plotting
### Catalogue map

In [ ]:
if 1:
    
    lomin = 98
    lomax = 108
    lamin = 20
    lamax = 34    
    
    fig = plt.figure(figsize=(20,16))
    m = Basemap(llcrnrlon=lomin,
                llcrnrlat=lamin,
                urcrnrlon=lomax,
                urcrnrlat=lamax,
                resolution='i',
                projection='tmerc',
                lon_0=106,
                lat_0=28)

    m.shadedrelief()
    
    parallels = numpy.arange(lamin, lamax, 5.)
    m.drawparallels(parallels,labels=[False,True,True,False], fontsize=16)
    meridians = numpy.arange(90, lomax, 5.)
    m.drawmeridians(meridians,labels=[True,False,False,True], fontsize=16)
    
    # Coordinate conversion
    idx = numpy.nonzero(flag!=0)[0]
    x, y = m(catalogue.data['longitude'], catalogue.data['latitude'])
    
    print idx.shape
    print x.shape

    # catalogue    
    x, y = m(catalogue.data['longitude'], catalogue.data['latitude'])
    plt.plot(x, y, 'o', linewidth=6, alpha=0.4, color='blue')
    #plt.scatter(x, y, catalogue.data['magnitude'], marker='o', linewidth=0.5, alpha=0.4, edgecolors='none')
    
    x, y = m(catalogue_original.data['longitude'], catalogue_original.data['latitude'])
    plt.plot(x[idx], y[idx], 'x', linewidth=.5, alpha=0.8, color='red') 
    
    

## Catalogue completeness analysis

In [ ]:
config = {'magnitude_bin': .5, 'time_bin': 10., 'increment_lock': True}
stepp = Stepp1971()
compl_table = stepp.completeness(catalogue, config)          
subprocess.call('rm aa', shell=True)
#create_stepp_plot(stepp, 'aa')
subprocess.call('rm aa', shell=True)
# info
print 'Maximum magnitude is: ', max(catalogue.data['magnitude'])
print 'The catalogue contains %d events' % (catalogue.get_number_events())
print 'Completeness table: \n'
# Display html
for line in compl_table:
    print '%.2f, %.2f' % (line[0], line[1])

In [ ]:
plot_magnitude_time_density(catalogue, mag_int=0.1, time_int=2.5, normalisation=True, 
                            bootstrap=0, completeness=compl_table)

In [ ]:
idx = numpy.nonzero(compl_table[:,1] < numpy.max(catalogue.data['magnitude']))
weichert_config = {'magnitude_interval': 0.1, 
                   'reference_magnitude': 0.0}
weichert = Weichert()
bval_wei, sigmab, aval_wei, sigmaa = weichert.calculate(catalogue, weichert_config, completeness=compl_table)
print 'bval: %.3f (sigma=%.3f)' % (bval_wei, sigmab)
print 'aval: %.3f (sigma=%.3f)' % (aval_wei, sigmaa)

In [ ]:
from hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model
#plot_trunc_gr_model(aval_wei, bval_wei, 5.0, 9.5, 0.1, catalogue=catalogue,
#        completeness=compl_table)

## Saving catalogue pickle file and updating the project 

In [ ]:
model_lab = re.sub('\s', '_', oqtkp.models[model_id].name).lower()
output_path = os.path.join(oqtkp.directory, '%s_catalogue.pkl' % model_lab)
fou = open(output_path,'wb') 
pickle.dump(catalogue, fou)
fou.close()
print 'Catalogue dataset saved into file: \n%s' % (output_path)

### Saving completeness into hdf5 file

In [ ]:
filename = os.path.join(oqtkp.directory, oqtkp.compl_hdf5_filename)
fhdf5 = h5py.File(filename, 'a')
print 'Saving data into %s' % (filename)
dataset_name = 'whole_catalogue'
# Update/create group
if model_id in fhdf5.keys():
    print 'Group exists %s ' % (model_id)
    grp = fhdf5[model_id]
else:
    print 'Creating group: %s' % (model_id)
    grp = fhdf5.create_group(model_id)
# Update/create dataset
if dataset_name in grp:
    del fhdf5[model_id][dataset_name]
    print 'Updating dataset: %s' % (dataset_name)
    dataset = grp.create_dataset(dataset_name, data=compl_table)
else:
    print 'Creating dataset: %s' % (dataset_name)
    dataset = grp.create_dataset(dataset_name, data=compl_table)
fhdf5.close()

### Saving model

In [ ]:
oqtkp.models[model_id].declustered_catalogue_pickle_filename = os.path.relpath(output_path, oqtkp.directory)
oqtkp.save()
print oqtkp.models[model_id].declustered_catalogue_pickle_filename

In [ ]:
model = oqtkp.models[model_id]
model.get_info()

In [ ]:
print completed execution